In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv', index_col = 0)
float_df = train.drop(['target'], axis = 1).select_dtypes(float)
int_df = train.drop(['target'], axis = 1).select_dtypes(int)

mean_fl = float_df.mean(axis = 1)
std_fl = float_df.std(axis = 1)
mean_int = int_df.mean(axis = 1)
std_int = int_df.std(axis = 1)
train['mean_fl'] = mean_fl
train['mean_int'] = mean_int
train['std_fl'] = std_fl
train['std_int'] = std_int
train

In [ ]:
!pip install -U lightautoml

In [ ]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from lightautoml.tasks.common_metric import roc_auc_score

N_THREADS = 8
N_FOLDS = 5
TIMEOUT = 3600*2
RANDOM_STATE = 42

In [ ]:
task = Task('binary', metric= roc_auc_score)
roles = {
    'target' : 'target',
    'drop' : 'id'
}
#automl = TabularAutoML(task=task, general_params={'use_algos': [['linear_l2','cb','lgb_tuned'],['lgb', 'cb_tuned']]})

automl = TabularAutoML(task = task,
                       timeout = TIMEOUT,
                       general_params={'use_algos': [['linear_l2', 'lgb', 'cb_tuned'],['cb','lgb_tuned']]})
                       #tuning_params = {'max_tuning_iter': 20, 'max_tuning_time': 30},
                       #lgb_params = {'default_params': {'num_threads': N_THREADS}}, verbose=0)

In [ ]:
oof_pred = automl.fit_predict(train, roles= roles)

In [ ]:
test = pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv', index_col = 0)
float_df = test.select_dtypes(float)
int_df = test.select_dtypes(int)

mean_fl = float_df.mean(axis = 1)
std_fl = float_df.std(axis = 1)
mean_int = int_df.mean(axis = 1)
std_int = int_df.std(axis = 1)
test['mean_fl'] = mean_fl
test['mean_int'] = mean_int
test['std_fl'] = std_fl
test['std_int'] = std_int
test

In [ ]:
test_pred = automl.predict(test)

In [ ]:
submiss = pd.read_csv('../input/tabular-playground-series-oct-2021/sample_submission.csv', index_col=0)
submiss['target'] = test_pred.data
submiss.to_csv('./my_submissv2.csv')